In [1]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import os

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 119s   
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# Convert class vectors to binary class matrices. (One-hot-encoded)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


### Iniciando o modelo

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

### Declare compiler

In [6]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


### Normalizando a imagem

In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [8]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=int(np.ceil(x_train.shape[0] / float(batch_size))),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)



Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 216s - loss: 1.8900 - acc: 0.3014 - val_loss: 1.5746 - val_acc: 0.4342
Epoch 2/100
1563/1563 [==============================] - 213s - loss: 1.5851 - acc: 0.4172 - val_loss: 1.3991 - val_acc: 0.4947
Epoch 3/100
1563/1563 [==============================] - 210s - loss: 1.4609 - acc: 0.4721 - val_loss: 1.2719 - val_acc: 0.5458
Epoch 4/100
1563/1563 [==============================] - 213s - loss: 1.3653 - acc: 0.5105 - val_loss: 1.2085 - val_acc: 0.5671
Epoch 5/100
1563/1563 [==============================] - 208s - loss: 1.2905 - acc: 0.5401 - val_loss: 1.1499 - val_acc: 0.6002
Epoch 6/100
1563/1563 [==============================] - 203s - loss: 1.2235 - acc: 0.5644 - val_loss: 1.0632 - val_acc: 0.6299
Epoch 7/100
1563/1563 [==============================] - 204s - loss: 1.1732 - acc: 0.5835 - val_loss: 1.0987 - val_acc: 0.6189
Epoch 8/100
1563/1563 [==============================] - 205s - loss:

1563/1563 [==============================] - 212s - loss: 0.7766 - acc: 0.7412 - val_loss: 0.6874 - val_acc: 0.7812
Epoch 65/100
1563/1563 [==============================] - 209s - loss: 0.7733 - acc: 0.7426 - val_loss: 0.7857 - val_acc: 0.7571
Epoch 66/100
1563/1563 [==============================] - 208s - loss: 0.7703 - acc: 0.7435 - val_loss: 0.6594 - val_acc: 0.7818
Epoch 67/100
1563/1563 [==============================] - 210s - loss: 0.7734 - acc: 0.7425 - val_loss: 0.6620 - val_acc: 0.7769
Epoch 68/100
1563/1563 [==============================] - 210s - loss: 0.7752 - acc: 0.7414 - val_loss: 0.6895 - val_acc: 0.7802
Epoch 69/100
1563/1563 [==============================] - 209s - loss: 0.7831 - acc: 0.7375 - val_loss: 0.6735 - val_acc: 0.7750
Epoch 70/100
1563/1563 [==============================] - 208s - loss: 0.7769 - acc: 0.7404 - val_loss: 0.7545 - val_acc: 0.7543
Epoch 71/100
1563/1563 [==============================] - 216s - loss: 0.7694 - acc: 0.7434 - val_loss: 0.7105

In [9]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)



Saved trained model at C:\Users\Tales\algoritmos_fcs\keras\tutorial\saved_models\keras_cifar10_trained_model.h5 


In [10]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 11s    
Test loss: 0.682266796303
Test accuracy: 0.7842
